In [1]:
import pandas as pd
import requests
import re

FULL_PATH = "C:/Users/caval/Documents/tech-challenge-01"

In [2]:
request = requests.get("https://www.wine-searcher.com/regions-rio+grande+do+sul?tab_F=mostpopular#winesortlist")
request.status_code

403

In [21]:
def format_wine_data(row, data_type):
    if (data_type == 'product'):
        return row.split('Brazil')[0][:-2]
    
    if (data_type == 'grape'):
        mixed_string = row.split('Brazil')[-1]
        return mixed_string[:re.search(r'\d', mixed_string).start()]
    
    if (data_type == 'score'):
        splitted_data = row.split('/')
        
        if ('100' in splitted_data[-1]):
            score = splitted_data[-2][-3:].strip()
            return int(score)
        else:
            return
    if (data_type == 'avg_price_liter'):
        price_750_ml = int(row[row.find("R$") - 5 : row.find("R$")].strip().replace(",", ""))
        liter = price_750_ml + (price_750_ml / 3)
        return float("{:.2f}".format(liter))

In [22]:
columns = ['product', 'grape', 'score', 'avg_price_liter']
dataset = pd.DataFrame(columns=columns)
dataset

,product,grape,score,avg_price_liter


In [23]:
for x in range(1, 6):
    file = open(f'{FULL_PATH}/scripts/{x}.html')
    data = pd.read_html(file.read())
    data = data[0]
    
    data = data.reset_index(drop=True)
    
    data.columns = ['temp1', 'temp2']
    
    for column in columns:
        data[column] = data['temp2'].apply(format_wine_data, data_type=column)
        
    data = data.drop(['temp1', 'temp2'], axis=1)
    
    dataset = pd.concat([dataset, data])

In [24]:
dataset = dataset.set_index('product')
dataset.to_csv(f'{FULL_PATH}/scripts/brazilian_wine_scores.csv')

In [19]:
dataset

,product,grape,score,avg_price_liter
0,"Vega Sicilia Unico Gran Reserva, Ribera del D...",Tempranillo,95,622.67
1,"Bodegas Zugober Belezos 50/50, Rioja DOCa",Rioja Red Blend,88,74.67
2,"R. Lopez de Heredia Vina Tondonia Reserva, Ri...",Rioja Red Blend,93,346.67
3,"La Rioja Alta S.A. Gran Reserva 904, Rioja DOCa",Rioja Red Blend,93,536.00
4,Marques de Murrieta Castillo Ygay Gran Reserv...,Rioja Red Blend,93,93.33
...,...,...,...,...
20,Bodegas Juan Gil 18 Meses Blue Label Monastre...,Monastrell,91,214.67
21,"Beronia Gran Reserva, Rioja DOCa,",Tempranillo,90,222.67
22,"Bodegas Toro Albala Don PX Gran Reserva, Mont...",Pedro Ximenez,91,437.33
23,"Torres Sangre de Toro Original, Catalonia",Carinena - Garnacha,85,65.33
